# Excel

Lendo a planilha Excel em [https://github.com/ocefpaf/CursoPython_AlunasJu/issues/5](https://github.com/ocefpaf/CursoPython_AlunasJu/issues/5).

Parece que tem dados depois de espaços em branco na mesma coluna. Abrindo a planilha parece que os dados de interesse terminam em ano 1732, amostra `AMCO564`, linha 79.

In [ ]:
import pandas as pd


skiprows = 2

df = pd.read_excel(
    "TOC.xlsx",
    skiprows=skiprows,
    nrows=79 - skiprows - 1,  # header + extra/blanks rows.
)

df.tail()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(6, 7), sharey=True)

ax0.plot(df["Rock Eval"].values, df["Ano"], ".")
ax1.plot(df["LECO"].values, df["Ano"], ".")

In [ ]:
import seaborn


seaborn.set(style="darkgrid")

g = seaborn.jointplot("Rock Eval", "LECO", data=df, kind="reg", height=7,)

# NetCDF

In [ ]:
from netCDF4 import Dataset


nc = Dataset("woa01an.nc")
nc

In [ ]:
import numpy as np

lon = nc["WOA01_LONX"][:]
lat = nc["WOA01_LATX"][:]
temp = nc["WOA01_MEAN_TEMP"][:]
sal = nc["WOA01_MEAN_PSAL"][:]
depth = nc["WOA01_DEPH"][:]

maskx = np.logical_and(lon < -5.5, lon > -45.5)
masky = np.logical_and(lat < 0, lat > -45)

temp = temp[masky, ...][:, maskx, :].mean(axis=0).mean(axis=0)
sal = sal[masky, ...][:, maskx, :].mean(axis=0).mean(axis=0)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(ncols=2, sharey=True, figsize=(12, 8))

ax0.invert_yaxis()

ax0.plot(temp, depth)
ax0.set_xlabel(u"Temperatura [\u00B0C]")
ax0.set_ylabel("Profundidade [m]")

ax1.plot(sal, depth)
ax1.set_xlabel(r"Salinidade [g kg$^{-1}$]")

# OPeNDAP

In [ ]:
from netCDF4 import Dataset


url = "http://goosbrasil.org:8080/pirata/B19s34w.nc"
nc = Dataset(url)

temp = nc["temperature"][:]
times = nc["time"]

In [ ]:
temp[temp <= -9999] = np.NaN

In [ ]:
import cftime
import nc_time_axis


t = cftime.num2date(times[:], times.units, calendar=times.calendar,)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, temp[:, 0], ".")

## Mesmo com xarray/iris CF - Climate Forecast

In [ ]:
import xarray as xr

ds = xr.open_dataset(url)
ds

In [ ]:
temp = ds["temperature"]

temp

In [ ]:
temp.sel(depth_t=1.0, time="2008").plot()

# CDS

https://cds.climate.copernicus.eu/api-how-to

In [ ]:
import cdsapi
import fsspec
import xarray as xr

try:
    c = cdsapi.Client()

    name = "reanalysis-era5-pressure-levels"
    request = {
        "year": "2008",
        "month": "01",
        "day": "01",
        "time": "12:00",
        "variable": "temperature",
        "pressure_level": "1000",
        "product_type": "reanalysis",
        "format": "netcdf",
    }

    r = c.retrieve(name, request, None)
    print(r.location)


    with fsspec.open(r.location) as f:
        ds = xr.open_dataset(f)

    temp = ds["t"].squeeze()
    temp.plot()
except Exception as e:
    print(e)

# ERDDAP

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(server="https://data.ioos.us/gliders/erddap", protocol="tabledap",)

In [ ]:
e.dataset_id = "whoi_406-20160902T1700"

e.constraints = {
    "time>=": "2016-07-10T00:00:00Z",
    "time<=": "2017-02-10T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]


url = e.get_download_url()

print(url)

In [ ]:
import pandas as pd


df = e.to_pandas(index_col="time (UTC)", parse_dates=True,).dropna()

df.head()

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(17, 2))
cs = ax.scatter(
    df.index,
    df["depth (m)"],
    s=15,
    c=df["temperature (Celsius)"],
    marker="o",
    edgecolor="none",
)

ax.invert_yaxis()
ax.set_xlim(df.index[0], df.index[-1])
xfmt = mdates.DateFormatter("%H:%Mh\n%d-%b")
ax.xaxis.set_major_formatter(xfmt)

cbar = fig.colorbar(cs, orientation="vertical", extend="both")
cbar.ax.set_ylabel("Temperature ($^\circ$C)")
ax.set_ylabel("Depth (m)")

One server to rule them all!

In [ ]:
e.get_download_url(response="opendap")

In [ ]:
e.get_download_url(response="mat")

In [ ]:
e.get_download_url(response="csv")

In [ ]:
e.get_download_url(response="json")

In [ ]:
e.get_download_url(response="ncCF")

In [ ]:
e.get_download_url(response="ncCFHeader")

# Data Access Gallery

https://ioos.github.io/notebooks_demos/code_gallery/

Exercício: Encontre um set de dados em

https://coastwatch.pfeg.noaa.gov/erddap/info/index.html

e, usando o `erddapy` faça um gráfico com o dado escolhido.